# 데이터 불러오기 및 구분

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display as dp
import warnings
import matplotlib
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv') # 사용안함
sample_submission = pd.read_csv('../data/sample_submission.csv')

dp(train_df.head())
dp(test_df.head())
dp(sample_submission)

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,TRAIN_00003,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,TRAIN_00004,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2


,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,TEST_00000,9,금요일,5,927,28.0,1.570654,19.625000,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,차도
1,TEST_00001,5,수요일,3,926,28.0,1.712457,21.444444,0.0,0.0,175.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,식당
2,TEST_00002,5,월요일,6,1437,33.0,0.447496,25.200000,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
3,TEST_00003,11,화요일,1,1739,31.0,0.878585,0.000000,0.0,0.0,285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
4,TEST_00004,10,목요일,10,830,15.0,0.496423,26.142857,0.0,0.0,95.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지


,ID,TARGET
0,TEST_00000,0
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0
...,...,...
17284,TEST_17284,0
17285,TEST_17285,0
17286,TEST_17286,0
17287,TEST_17287,0


In [2]:
# 추후 삭제
################################
# 연속형/범주형 구분
train_df['범죄발생지'].value_counts()
def type_change(df,cols):
    for col in cols:
        df[col]  = df[col].astype('category')

categorical = ['월', '요일', '시간', '소관경찰서', '소관지역', '안개', '짙은안개', '번개', '진눈깨비', '서리', '연기/연무', '눈날림', '범죄발생지']
continuous = list(set(train_df.iloc[:,1:-1].columns)-set(categorical))

type_change(train_df, categorical)
type_change(train_df, ['TARGET'])

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84406 entries, 0 to 84405
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   ID       84406 non-null  object  
 1   월        84406 non-null  category
 2   요일       84406 non-null  category
 3   시간       84406 non-null  category
 4   소관경찰서    84406 non-null  category
 5   소관지역     84406 non-null  category
 6   사건발생거리   84406 non-null  float64 
 7   강수량(mm)  84406 non-null  float64 
 8   강설량(mm)  84406 non-null  float64 
 9   적설량(cm)  84406 non-null  float64 
 10  풍향       84406 non-null  float64 
 11  안개       84406 non-null  category
 12  짙은안개     84406 non-null  category
 13  번개       84406 non-null  category
 14  진눈깨비     84406 non-null  category
 15  서리       84406 non-null  category
 16  연기/연무    84406 non-null  category
 17  눈날림      84406 non-null  category
 18  범죄발생지    84406 non-null  category
 19  TARGET   84406 non-null  category
dtypes: category(14), float64(5),

In [3]:
# 시계열 데이터가 아니므로 sklearn의 train_test_split 사용
from sklearn.model_selection import train_test_split
X = train_df.iloc[:,1:-1] # ID 컬럼은 필요없음.
y = train_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state=9999)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=9999)
print(f'''
train({X_train.shape[0]/X.shape[0]*100:.3f})% : X => {X_train.shape}
val({X_val.shape[0]/X.shape[0]*100:.3f})% : X => {X_val.shape}
test({X_test.shape[0]/X.shape[0]*100:.3f})% : X => {X_test.shape}''')


train(59.999)% : X => (50643, 18)
val(20.000)% : X => (16881, 18)
test(20.001)% : X => (16882, 18)


In [4]:
# 새롭게 사용하게 될 데이터셋 정의
new_train = pd.concat([X_train, y_train], axis = 1)
new_validation = pd.concat([X_val, y_val], axis = 1)
new_test = pd.concat([X_test, y_test], axis = 1)

In [5]:
# new_train.to_csv('../data/new_train.csv')
# new_validation.to_csv('../data/new_validation.csv')
# new_test.to_csv('../data/new_test.csv')

# 개별 컬럼 단위의 이상치 (IQR)

In [6]:
continuous

['적설량(cm)', '사건발생거리', '풍향', '강설량(mm)', '강수량(mm)']

In [7]:
def find_each_outlier(df,col):
    q3 = df[col].quantile(0.75)
    q1 = df[col].quantile(0.25)
    IQR = q3-q1
    return (df[col]<q1-1.5*IQR) | (df[col]>q3+1.5*IQR)

## 삭제

In [8]:
results={}
for col in continuous:
    result = X_train[find_each_outlier(X_train, col)] 
#     print(f'{col} 이상치 : {X_train.shape[0]-result.shape[0]}')
#     dp(result.head(3))
    results[col] = result
    
# 이상치로 판정된 개별 변수의 모든 측정치 찾기
normal_idx = set()
for key in list(results.keys()):
    idx = set(results[key].index)
    normal_idx.update(idx)
    
len(list(normal_idx)) # 14964개의 인덱스가 이상치로 판정됨.
X_train.drop(list(normal_idx)) # 관측치의 모든 개별 변수가 정상치인 데이터

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
35583,11,수요일,11,427,13.0,2.042390,10.857143,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,차도
65048,8,수요일,12,829,29.0,0.532356,0.000000,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,인도
4526,5,월요일,6,1037,31.0,2.036113,0.000000,0.0,0.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
7275,5,일요일,12,538,10.0,1.222209,11.555556,0.0,0.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
61028,6,화요일,7,2337,53.0,2.202734,0.000000,0.0,0.0,110.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주차장
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,8,일요일,8,246,9.0,2.457688,3.500000,0.0,0.0,220.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,차도
20082,4,수요일,4,237,10.0,0.564908,0.000000,0.0,0.0,285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
61034,10,수요일,11,1029,32.0,2.646083,42.500000,0.0,0.0,170.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,주거지
57513,6,수요일,4,1538,49.0,1.195191,1.000000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도


## 이상치에 가중치 적용


데이터 포인트 간의 거리를 계산한 후 거리의 역수를 가중치로 적용하였다.

In [10]:
iqr_df = X_train.copy()
for col in continuous:
    result = find_each_outlier(iqr_df,col)
    distance=np.abs(iqr_df.loc[result,col]-X_train.loc[~result,col].median()) 
    weight = 1/(1+distance) # 이상치가 중앙값에 가까우면 큰 가중치를, 멀어진다면 작은 가중치를
    iqr_df.loc[result,col] = iqr_df.loc[result,col] * weight 
    
iqr_df

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
35208,2,월요일,5,2440,39.0,1.967578,35.166667,0.985240,0.997241,205.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,주거지
36491,11,목요일,2,2449,30.0,2.862873,0.983491,0.000000,0.000000,265.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
48108,6,월요일,1,1336,5.0,0.437663,0.997052,0.000000,0.000000,310.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,편의점
35583,11,수요일,11,427,13.0,2.042390,10.857143,0.000000,0.000000,290.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,차도
65048,8,수요일,12,829,29.0,0.532356,0.000000,0.000000,0.000000,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,인도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39711,1,금요일,2,1037,41.0,2.640971,19.750000,0.891892,0.975410,45.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,차도
27764,1,화요일,4,1947,44.0,1.450236,0.000000,0.000000,0.909091,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
61034,10,수요일,11,1029,32.0,2.646083,42.500000,0.000000,0.000000,170.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,주거지
57513,6,수요일,4,1538,49.0,1.195191,1.000000,0.000000,0.000000,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도


In [11]:
# # # 1번만 실행
# IQR_weight = pd.concat([iqr_df,y_train[iqr_df.index]],axis = 1)
# IQR_weight.to_csv('../data/IQR_weight.csv')

___

# Isolation Forest로 탐지한 이상치

In [12]:
from sklearn.ensemble import IsolationForest

np.random.seed(9999)
model = IsolationForest()
X_continuous = X_train[continuous]
model = model.fit(X_continuous)
result = model.predict(X_continuous)
result

array([-1,  1, -1, ...,  1,  1,  1])

In [13]:
outliers=[]
for i, outlier in enumerate(result):
    if outlier== -1: outliers.append(i)
    else : continue
len(outliers) # 위의 방법보다 훨씬 적은 이상치
print(f'이상치 비율 : {len(outliers)/train_df.count(axis=0)[0]*100:.2f}%')

이상치 비율 : 8.06%


In [14]:
# 직접 확인
X_train.iloc[outliers,:]

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
35208,2,월요일,5,2440,39.0,1.967578,35.166667,66.750000,361.400000,205.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,주거지
48108,6,월요일,1,1336,5.0,0.437663,338.200000,0.000000,0.000000,310.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,편의점
67859,1,수요일,6,1946,44.0,0.470217,28.875000,34.000000,144.000000,345.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,약국
13884,1,금요일,6,1928,52.0,1.267569,0.000000,0.000000,123.600000,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
44798,2,일요일,3,847,20.0,2.029694,16.500000,4.285714,434.600000,175.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,차도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65995,3,월요일,1,749,22.0,3.514836,9.125000,6.571429,0.000000,90.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,주거지
59038,2,일요일,12,1030,32.0,2.377448,10.125000,10.200000,272.600000,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
52566,2,토요일,10,426,13.0,2.738335,25.142857,45.500000,309.400000,325.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,주거지
83498,6,일요일,8,936,28.0,0.437052,379.333333,0.000000,0.000000,235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지


## 삭제

In [15]:
# # outliers가 아닌 관측치만 확인하면 됨.
# outliers_index = new_train.iloc[outliers,:].index # 실제 인덱스명 통해 drop하기 위해
# Iso_Forest_delete = new_train.drop(outliers_index)
# Iso_Forest_delete.to_csv('../data/Isolation_Forest_delete.csv') # 1번만 실행

## 이상치에 가중치 적용

Isolation Forest로 발견한 이상치는 각 관측치별로 **이상치 점수가 존재**  
$\Rightarrow$ 이 값을 가중치로 적용함.

In [16]:
scores = model.decision_function(X_continuous)
scores

array([-0.16601867,  0.07363077, -0.1250395 , ...,  0.09911389,
        0.15035841,  0.12988652])

In [17]:
# 가중치에는 최소-최대 스케일을 진행.
scores.min(), scores.max()

(-0.3015213056792482, 0.16270418374258866)

In [18]:
# 이상치일수록 더 낮은 가중치를 받게 됨.
IForest_df = X_train.copy()
weight = (scores-scores.min())/(scores.max()-scores.min()) # 점수를 0~1 사이로
IForest_continuous = pd.DataFrame(X_continuous*weight.reshape(-1,1), columns = continuous) # (84406,5) (84406,1)
dp(IForest_continuous)

IForest_df.loc[:,continuous] = IForest_continuous
IForest_df

,적설량(cm),사건발생거리,풍향,강설량(mm),강수량(mm)
35208,105.488933,0.574316,59.837386,19.483637,10.264787
36491,0.000000,2.313558,214.153038,0.000000,48.141141
48108,0.000000,0.166384,117.850828,0.000000,128.571452
35583,0.000000,1.890658,268.455451,0.000000,10.050549
65048,0.000000,0.466020,78.785241,0.000000,0.000000
...,...,...,...,...,...
39711,21.305129,1.418477,24.169684,4.431109,10.607806
27764,9.143750,1.326059,228.593752,0.000000,0.000000
61034,0.000000,2.283618,146.713150,0.000000,36.678287
57513,0.000000,1.163406,194.681131,0.000000,0.973406


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
35208,2,월요일,5,2440,39.0,0.574316,10.264787,19.483637,105.488933,59.837386,1.0,0.0,0.0,0.0,0.0,1.0,1.0,주거지
36491,11,목요일,2,2449,30.0,2.313558,48.141141,0.000000,0.000000,214.153038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
48108,6,월요일,1,1336,5.0,0.166384,128.571452,0.000000,0.000000,117.850828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,편의점
35583,11,수요일,11,427,13.0,1.890658,10.050549,0.000000,0.000000,268.455451,1.0,0.0,0.0,0.0,0.0,1.0,0.0,차도
65048,8,수요일,12,829,29.0,0.466020,0.000000,0.000000,0.000000,78.785241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,인도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39711,1,금요일,2,1037,41.0,1.418477,10.607806,4.431109,21.305129,24.169684,1.0,0.0,0.0,1.0,1.0,1.0,0.0,차도
27764,1,화요일,4,1947,44.0,1.326059,0.000000,0.000000,9.143750,228.593752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지
61034,10,수요일,11,1029,32.0,2.283618,36.678287,0.000000,0.000000,146.713150,1.0,0.0,1.0,0.0,0.0,0.0,0.0,주거지
57513,6,수요일,4,1538,49.0,1.163406,0.973406,0.000000,0.000000,194.681131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도


In [19]:
# # # 1번만 실행
# Isolation_Forest_weight = IForest_df
# Isolation_Forest_weight = pd.concat([Isolation_Forest_weight, y_train[Isolation_Forest_weight.index]], axis = 1)
# Isolation_Forest_weight.to_csv('../data/Isolation_Forest_weight.csv')

___

# DBScan

거리에 의존하기 때문에 scaler를 사용해야함.   
$\Rightarrow$ 원 데이터를 그대로 보지않고 스케일 후 탐지하는 것은 바람직하지 않다고 판단하여 이 방법은 사용하지 않음.

In [20]:
# from sklearn,cluster import DBSCAN
# # eps : 어떤 데이터 포인트로부터 다른 데이터 포인트까지의 최대 거리 이 거리 내의 데이터는 같은 군집
# # min_samples : 군집 형성을 위한 최소 데이터 포인트 수
# model = DBSCAN(eps=0.3, min_samples)

___